# Microring resonator metrics

## Theory

### Objective

This notebook computes metrics of interest for a symmetric add-drop microring resonator from two sets of simulation.
* An eigenmode simulation for the wavelength-dependent complex effective index of the bent waveguide.
* A finite difference time domain simulation for the coupling coefficient of the straight waveguide-curved waveguide coupler region.

We provide such results for a ring radius of 8 microns and waveguide width $500$ nm under respective directories. Varying bus-ring gaps from 200 nm to 500 nm are considered for the coupler. The waveguide geometry is also the standard silicon clad with SiO2, with 220 nm core thickness, and 90 nm slab thickness.  Another input is the excess ring loss $\alpha_{ext}$, which here incorporates foundry-reported propagation loss $\sim 1.5$ dB/cm; a more complete model would also include loss due to free-carrier dispersion in doped waveguides.

### Theory

The two characteristic parameters are the self-coupling coefficient $r$ (thru port amplitude) and round-trip loss coefficient $a \rightarrow ra$ ; the extra factor or $r$ comes from the symmetric coupling at the drop bus during the round trip.

An intermediate quantity, the free spectral range $$ \text{FSR} = \frac{\lambda^2}{2\pi R}\left( n_{eff}(\lambda) - \lambda\frac{\partial n_{eff}}{\partial \lambda} \right)^{-1} $$ is required to extra the final desired derived quantities.

The derived quantities desired are the finesse (also related to full width half maximum) 
$$\mathcal{F} = \pi\arccos\left(\frac{2ar^2}{1+a^2r^4}\right) = \frac{\text{FSR}}{\text{FWHM}} $$
Q-factor 
$$ Q = \frac{\lambda\mathcal{F}}{\text{FSR}}, $$ 
thru extinction ratio 
$$ ER_{thru} = \frac{(a+1)^2}{(a-1)^2}\frac{(1-r^2a)^2}{(1+r^2a)^2}, $$
drop extinction ratio $$ ER_{drop} = \frac{(1 + r^2a)^2}{(1 - r^2a)^2}, $$
and insertion loss
$$ IL = \frac{r^2(1+a)^2}{(1+r^2a)^2}$$
of the ring.

### Workflow

* For every radius $R$ (for now, only 8 microns is considered) :
    * An eigensolver simulation is performed to extract the effective index and loss coefficient of the ring's bent waveguide
        * From the loss $\alpha (\lambda)$ in dB/m, a wavelength-dependent fit for the loss coefficient $a$ is obtained via $a^2 = e^{-\alpha \cdot 2\pi R}$.
        * From the effective index, the group index is obtained; from the group index, the free spectral range can be fitted as a function of wavelength
    * For every geometry (bus-ring gap and bus width) :
        * A FDTD simulation on the coupler section is performed to extract the S-parameters. $r$ is then simply $T_{thru}$ (amplitude transmission coefficient, or square root of power transmission coefficient). 

The convention is
* Simulation point : solid marker
* Fit : line

A line alone means data extracted from earlier fits (used for plotting of final quantities).

### References:

Bogaerts, W., De Heyn, P., Van Vaerenbergh, T., De Vos, K., Kumar Selvaraja, S., Claes, T., … Baets, R. (2012). Silicon microring resonators. Laser & Photonics Reviews, 6(1), 47–73. https://doi.org/10.1002/lpor.201100017

McKinnon, W. R., Xu, D. X., Storey, C., Post, E., Densmore, A., Delâge, A., … Janz, S. (2009). Extracting coupling and loss coefficients from a ring resonator. Optics Express, 17(21), 18971. https://doi.org/10.1364/OE.17.018971.

## Code

### Setup

In [1]:
from IPython.display import HTML, display

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bokeh.io import output_notebook, show
from bokeh.plotting import figure, output_file
from bokeh.layouts import row
from bokeh.models import ColumnDataSource
from bokeh.palettes import viridis
import itertools  
from pathlib import Path
from scipy.optimize import curve_fit
output_notebook()

Loading BokehJS ...

### Inputs

In [5]:
# FDTD coupler simulation directory
fdtd_dir = 'FDTD'
# MODE bent waveguide simulation directory
mode_dir = 'MODE'

# Ring radius
R = 8 # microns
# Waveguide widths
w = 500 # nm

# Extra loss channels
# Doped waveguide propagation
prop_loss = 5 # dB/cm

### Load data

The MODE simulation is in a text file with columns *f,neff(real),neff(imag),loss,beta(real),beta(imag),overlap,vg,D,Z0(real),Z0(imag)*

All FDTD simulation files and put into a Pandas dataframe with columns *['wavelength', 'frequency', 'Re_Sdrop', 'Im_Sdrop', 'Abs_Sdrop', 'Angle_Sdrop', 'Re_Sthru', 'Im_Sthru', 'Abs_Sthru', 'Angle_Sthru', 'r', 'd', 'w']*. 

In [6]:
# Setup
from pprint import pprint
# Load MODE data
path_MODE = Path('MODE/R_8mum_Tslab_90nm.txt')
ring = np.loadtxt(path_MODE,skiprows=1,delimiter=',')

# Load FDTD data
path_FDTD = Path('FDTD')
df = pd.DataFrame()
def searching_all_files(directory):
    dirpath = Path(directory)
    assert(dirpath.is_dir())
    file_list = []
    for x in dirpath.iterdir():
        if x.is_file():
            file_list.append(x)
        elif x.is_dir():
            file_list.extend(searching_all_files(x))
    return file_list
files = searching_all_files(path_FDTD)
for file in files:
    # Parse radius and gap
    filename = file.name
    if filename[-4:] == '.dat':
        parsed_filename = file.name[:-4].split('_')
        r = parsed_filename[2] # microns
        d = parsed_filename[4] # nm
        # if width is provided, add it
        if len(parsed_filename) == 7:
            w = parsed_filename[6] # nm
        else: # otherwise it was 500nm
            w = 500
        # Create local dataframe
        cur_df = pd.read_csv(file, header=None)
        cur_df['r'] = r
        cur_df['d'] = d
        cur_df['w'] = w
        # Append to final dataframe
        df = df.append(cur_df)
    else:
        continue
# Add missing keys to dataframe
df[0] = df[0]*1E6
df = df.rename(columns={0: "wavelength", 1: "frequency", 2: "Re_Sdrop", 3: "Im_Sdrop", 4: "Abs_Sdrop", 5: "Angle_Sdrop", 6: "Re_Sthru", 7: "Im_Sthru", 8: "Abs_Sthru", 9: "Angle_Sthru"})
print(list(df.columns.values))

['wavelength', 'frequency', 'Re_Sdrop', 'Im_Sdrop', 'Abs_Sdrop', 'Angle_Sdrop', 'Re_Sthru', 'Im_Sthru', 'Abs_Sthru', 'Angle_Sthru', 'r', 'd', 'w']


### MODE Analysis

The group index and loss coefficients are computed from the effective index (built-in Lumerical MODE) :

In [7]:
neff_ring_f = ring[:,0]
neff_ring_lambda = 299792458/neff_ring_f
neff_ring_re = ring[:,1]
neff_ring_im = ring[:,2]
loss_ring = ring[:,3]

colors = viridis(4) # itertools.cycle(viridis(2))

''' Effective index plots
p1 = figure(plot_width=400, plot_height=400, title='w=500mum, 90nm slab rib Re(neff)',tools="hover,crosshair,pan,box_zoom,wheel_zoom,undo,redo,reset,save",active_inspect="hover", active_drag="box_zoom", toolbar_location="right")
p1.line(neff_ring_lambda*1E6, neff_ring_re, line_width=2, color=colors[1],legend='r=8mum ring')
p1.scatter(neff_ring_lambda*1E6, neff_ring_re, line_width=2, color=colors[1],legend='r=8mum ring')
p1.xaxis.axis_label = "Wavelength (micron)"
p1.yaxis.axis_label = "Re(neff)"

p2 = figure(plot_width=400, plot_height=400, tools="hover,crosshair,pan,box_zoom,wheel_zoom,undo,redo,reset,save",active_inspect="hover", active_drag="box_zoom", toolbar_location="right")
p2.line(neff_ring_lambda*1E6, neff_ring_im, line_width=2, color=colors[1],legend='r=8mum ring')
p2.scatter(neff_ring_lambda*1E6, neff_ring_im, line_width=2, color=colors[1],legend='r=8mum ring')

p2.xaxis.axis_label = "Wavelength (micron)"
p2.yaxis.axis_label = "Im(neff)"

p2.legend.location = "top_left"

show(row(p1,p2))
'''

def deriv(x, y): 
    dx = np.gradient(x)
    dy = np.gradient(y)
    return dy/dx

ng_ring = neff_ring_re - neff_ring_lambda*deriv(neff_ring_lambda,neff_ring_re)
FSR = neff_ring_lambda**2/(2*np.pi*8E-6)*np.reciprocal(ng_ring)

p1 = figure(plot_width=400, plot_height=300, tools="hover,crosshair,pan,box_zoom,wheel_zoom,undo,redo,reset,save",active_inspect="hover", active_drag="box_zoom", toolbar_location="right")
p1.scatter(neff_ring_lambda*1E6, ng_ring, line_width=2, color=colors[1],legend='r=8mum ring')
p1.xaxis.axis_label = "Wavelength (micron)"
p1.yaxis.axis_label = "Group index"

p2 = figure(plot_width=400, plot_height=300, tools="hover,crosshair,pan,box_zoom,wheel_zoom,undo,redo,reset,save",active_inspect="hover", active_drag="box_zoom", toolbar_location="right")
p2.scatter(neff_ring_lambda*1E6, loss_ring*2*np.pi*8E-6, line_width=2, color=colors[1],legend='r=8mum ring')

p2.xaxis.axis_label = "Wavelength (micron)"
p2.yaxis.axis_label = "Round trip loss (dB)"

p2.legend.location = "top_left"

show(row(p1,p2))

From the group index, we can get the free spectral range via $$ \Delta \lambda_{FSR} = \frac{\lambda^2}{2\pi R}\left( n_{eff}(\lambda) - \lambda\frac{\partial n_{eff}}{\partial \lambda} \right)^{-1} $$

A linear trend (function `FSR_wav(wavelength)`) fits the data well :

In [8]:
# FSR from mode data :
p1 = figure(plot_width=800, plot_height=400,tools="hover,crosshair,pan,box_zoom,wheel_zoom,undo,redo,reset,save",active_inspect="hover", active_drag="box_zoom", toolbar_location="right")
p1.scatter(neff_ring_lambda*1E6, FSR, line_width=2, color="black",legend='r=8mum ring')
p1.xaxis.axis_label = "Wavelength (micron)"
p1.yaxis.axis_label = "FSR (m)"

neff_ring_lambda_microns = neff_ring_lambda*1E6

aFSR,bFSR = np.polyfit(neff_ring_lambda_microns, FSR, deg=1)
def FSR_wav(x, a=aFSR, b=bFSR):
    return b + a*x

p1.line(neff_ring_lambda_microns, FSR_wav(neff_ring_lambda_microns), line_width=2, color="green",legend=f'{aFSR:.4e}' + 'x + ' + f'{bFSR:.4e}')

p1.legend.location = "top_left"

show(p1)

From the reported MODE loss in dB/m, we get $a$ directly via $a^2 = e^{-\alpha \cdot 2\pi R}$ :

In [12]:
# Get a (only radius and ring-parameter dependent)
def dB_to_lin(x):
   return np.power(10, x/10)
a = np.sqrt(np.exp(-1*(dB_to_lin(prop_loss)*100*2*np.pi*8E-6)))
    
popt, pcov = curve_fit(lambda t,a,b: a*t + b,  neff_ring_lambda_microns,  a,  p0=(0, 1))
aa = popt[0]
ba = popt[1]
def a_wav(x, a=aa, b=ba):
    return a*x + b

pa = figure(plot_width=600, plot_height=400, tools="hover,crosshair,pan,box_zoom,wheel_zoom,undo,redo,reset,save",active_inspect="hover", active_drag="box_zoom", toolbar_location="right")
pa.scatter(neff_ring_lambda_microns, a, line_width=2, color="black")
pa.line(neff_ring_lambda_microns, a_wav(neff_ring_lambda_microns), line_width=2, color="green", legend= f'{aa:.4f}' + 'x + ' + f'{ba:.4f}')
pa.xaxis.axis_label = "Wavelength (microns)"
pa.yaxis.axis_label = "a"

show(pa)

## FDTD Analysis

The Pandas dataframe can be cut in arbitrary ways to outline the data differently. Here, we plot drop and thru power transmissions as a function of gap for various wavelengths. The 2 or 3-parameter exponential fits 

$$ |T_{drop}|^2 = e^{ad\cdot x+bd} $$

and

$$ |T_{thru}|^2 = At\cdot e^{Bt\cdot x} + Ct $$

A dataframe `df_ER` contains these fit parameters at the various simulation wavelengths.

In [13]:
# Obtain unique parameters for looping
lambdas_unique = df['wavelength'].unique()
rs_unique = df['r'].unique()
ds_unique = df['d'].unique()
ws_unique = df['w'].unique()

In [14]:
colors = itertools.cycle(viridis(len(lambdas_unique))) 
p3 = figure(plot_width=400, plot_height=400, y_axis_type="log", tools="hover,crosshair,pan,box_zoom,wheel_zoom,undo,redo,reset,save",active_inspect="hover", active_drag="box_zoom", toolbar_location="right")
p5 = figure(plot_width=400, plot_height=400, tools="hover,crosshair,pan,box_zoom,wheel_zoom,undo,redo,reset,save",active_inspect="hover", active_drag="box_zoom", toolbar_location="right")

p3.xaxis.axis_label = "Gap (nm)"
p3.yaxis.axis_label = "|Tdrop|^2"

p5.xaxis.axis_label = "Gap (nm)"
p5.yaxis.axis_label = "|Tthru|^2 = r^2"

wavelength_list = []
At_list = []
Bt_list = []
Ct_list = []
ad_list = []
bd_list = []

for wavelength, color in zip(lambdas_unique, colors):
    d_filtered = df.query('wavelength == @wavelength & w == 500')
    ydrop = np.float64(d_filtered['Abs_Sdrop'].values)**2
    ythru = np.float64(d_filtered['Abs_Sthru'].values)**2
    x = np.float64(d_filtered['d'].values)
    
    # Fit
    ad,bd = np.polyfit(x, np.log(ydrop), deg=1)
    fit_drop = np.exp(ad*x+bd)
    
    popt, pcov = curve_fit(lambda t,A,B,C: A*np.exp(B*t) + C,  x,  ythru,  p0=(np.exp(bd), ad, 1))
    At = popt[0]
    Bt = popt[1]
    Ct = popt[2]
    fit_thru = At*np.exp(Bt*x) + Ct
    
    wavelength_list.append(wavelength)
    At_list.append(At)
    Bt_list.append(Bt)
    Ct_list.append(Ct)
    ad_list.append(ad)
    bd_list.append(bd)
    
    p3.scatter(x, ydrop, line_width=2, color=color, legend='lambda='+str(wavelength)+'mum')
    p3.line(x, fit_drop, line_width=2, color=color, alpha=0.5,legend='lambda='+str(wavelength)+'mum')
    
    p5.scatter(x, ythru, line_width=2, color=color, legend='lambda='+str(wavelength)+'mum')
    p5.line(x, fit_thru, line_width=2, color=color, alpha=0.5,legend='lambda='+str(wavelength)+'mum')

p3.legend.visible=False
p5.legend.visible=False

df_ER = pd.DataFrame({'wavelength': wavelength_list, 
                   'At': At_list, 
                   'Bt': Bt_list, 
                   'Ct': Ct_list, 
                   'ad': ad_list, 
                   'bd': bd_list})
del At_list, Bt_list, Ct_list, ad_list, bd_list
print(df_ER)

show(row(p3,p5))

    wavelength        At        Bt        Ct        ad        bd
0      1.60000 -0.922802 -0.010130  1.003558 -0.011001  0.028795
1      1.59557 -0.914448 -0.010201  1.003373 -0.011061  0.018481
2      1.59116 -0.906115 -0.010270  1.003198 -0.011120  0.008049
3      1.58678 -0.897816 -0.010340  1.003038 -0.011179 -0.002490
4      1.58242 -0.889587 -0.010409  1.002889 -0.011238 -0.013127
5      1.57808 -0.881413 -0.010478  1.002756 -0.011296 -0.023846
6      1.57377 -0.873288 -0.010546  1.002638 -0.011354 -0.034638
7      1.56948 -0.865193 -0.010614  1.002535 -0.011411 -0.045496
8      1.56522 -0.857106 -0.010681  1.002444 -0.011469 -0.056397
9      1.56098 -0.848982 -0.010748  1.002358 -0.011526 -0.067330
10     1.55676 -0.840822 -0.010813  1.002276 -0.011583 -0.078280
11     1.55256 -0.832588 -0.010877  1.002197 -0.011640 -0.089246
12     1.54839 -0.824311 -0.010940  1.002119 -0.011697 -0.100227
13     1.54424 -0.816015 -0.011002  1.002042 -0.011754 -0.111212
14     1.54011 -0.807747 

These are interpolated in this next step to consider arbitrary wavelengths. It is found that a quartic fit captures all possible variation within the range of gaps considered (zoom as needed) :

In [15]:
# Select some data and plot it
colors = itertools.cycle(viridis(len(ds_unique)))
p3 = figure(plot_width=400, plot_height=400, title='r='+str(r)+' microns, w=500nm',tools="hover,crosshair,pan,box_zoom,wheel_zoom,undo,redo,reset,save",active_inspect="hover", active_drag="box_zoom", toolbar_location="right")
p5 = figure(plot_width=400, plot_height=400, title='r='+str(r)+' microns, w=500nm',tools="hover,crosshair,pan,box_zoom,wheel_zoom,undo,redo,reset,save",active_inspect="hover", active_drag="box_zoom", toolbar_location="right")

p3.xaxis.axis_label = "Wavelength (mcrons)"
p3.yaxis.axis_label = "|Tdrop|^2"

p5.xaxis.axis_label = "Wavelength (microns)"
p5.yaxis.axis_label = "|Tthru|^2"

for d, color in zip(ds_unique, colors):
    d_filtered = df.query('d == @d')
    ydrop = np.float64(d_filtered['Abs_Sdrop'].values)**2
    ythru = np.float64(d_filtered['Abs_Sthru'].values)**2
    x = np.float64(d_filtered['wavelength'].values)
    
    # Quartic fits
    popt, pcov = curve_fit(lambda t,a,b,c,d,e: a*t**4 + b*t**3 + c*t**2 + d*t + e,  x,  ydrop,  p0=(1, 1, 1, 1, 1))
    ad = popt[0]
    bd = popt[1]
    cd = popt[2]
    dd = popt[3]
    ed = popt[4]
    fit_drop = ad*x**4 + bd*x**3 + cd*x**2 + dd*x + ed
    
    popt, pcov = curve_fit(lambda t,a,b,c,d,e: a*t**4 + b*t**3 + c*t**2 + d*t + e,  x,  ythru,  p0=(1, 1, 1, 1, 1))
    at = popt[0]
    bt = popt[1]
    ct = popt[2]
    dt = popt[3]
    et = popt[4]
    fit_thru = at*x**4 + bt*x**3 + ct*x**2 + dt*x + et 
    
    p3.scatter(x, ydrop, line_width=2, color=color, legend='d='+str(d)+'nm')
    p3.line(x, fit_drop, line_width=2, color=color)
    p5.scatter(x, ythru, line_width=2, color=color, legend='d='+str(d)+'nm')
    p5.line(x, fit_thru, line_width=2, color=color)
    
p3.legend.visible=False
p5.legend.visible=False

# show
show(row(p3,p5))

This results in a final function call which computes reliable wavelength fits on the fly at given gaps. `ring_specs(gap, wavelength_in)` takes in any gap between 100 nm and 500 nm and any wavelength between 1.5 microns and 1.6 microns and returns a 1D array containing gap, wavelength, $r$, $a$, FSR, $\mathcal{F}$, Q, FWHM, ERt (in dB), ERd (in dB), and IL :

In [16]:
def lin_to_dB(x):
    return 10*np.log10(x)

def ring_specs(gap, wavelength_in):
    # Gap in nm
    # Wavelength in microns
    
    # For every data wavelength in this gap, extract transmission data
    wavelength_list = []
    r_list = []
    for wavelength in lambdas_unique:
        wavelength_list.append(wavelength)
        df_lambda = df_ER.query('wavelength == @wavelength')
        At = np.float64(df_lambda["At"].values)
        Bt = np.float64(df_lambda["Bt"].values)
        Ct = np.float64(df_lambda["Ct"].values)
        ad = np.float64(df_lambda["ad"].values)
        bd = np.float64(df_lambda["bd"].values)
        # Power transmission (DO SQUARE ROOT HERE)
        r_list.append(np.sqrt(At*np.exp(Bt*gap) + Ct))
    df_ER_gap = pd.DataFrame({'wavelength': wavelength_list, 'r': r_list})
    del wavelength_list, r_list

    # Fits
    x = df_ER_gap["wavelength"].values

    trans_coeff = df_ER_gap["r"].values.flatten()
    popt, pcov = curve_fit(lambda t,a,b,c,d,e: a*t**4 + b*t**3 + c*t**2 + d*t + e,  x,  trans_coeff,  p0=(1, 1, 1, 1, 1))
    ar = popt[0]
    br = popt[1]
    cr = popt[2]
    dr = popt[3]
    er = popt[4]
    def r_wav(x, a=ar, b=br, c=cr, d=dr, e=er):
        return a*x**4 + b*x**3 + c*x**2 + d*x + e

    # To return :
    r_calc= r_wav(wavelength_in)
    a_calc = a_wav(wavelength_in)
    FSR_calc = FSR_wav(wavelength_in)
    finesse = np.pi*np.reciprocal(np.arccos((2*a_calc*r_calc**2)/(1+a_calc**2*r_calc**4)))
    qfactor = wavelength_in*1E-6*finesse/FSR_calc # all lengths in meters
    FWHM = FSR_calc/finesse
    ERt = (a_calc+1)**2/(a_calc-1)**2*(1-r_calc**2*a_calc)**2/(1+r_calc**2*a_calc)**2
    ERd = (1+r_calc**2*a_calc)**2/(1-r_calc**2*a_calc)**2
    IL = r_calc**2*(1+a_calc)**2/(1 + r_calc**2*a_calc)**2
    
    return gap, wavelength_in, r_calc, a_calc, FSR_calc, finesse, qfactor, FWHM, lin_to_dB(ERt), lin_to_dB(ERd), IL

Using this to compute these quantities for various gaps between 200 and 500nm at different wavelengths near 1550nm :

In [17]:
gaps_toPlot = np.linspace(200,500,50)
wavelengths_toPlot = np.linspace(1.5,1.6,11)

results_list = []
for gap in gaps_toPlot:
    for wavelength in wavelengths_toPlot:
        gaps_list = []
        wavelengths_list = []
        results_list.append(ring_specs(gap, wavelength))
results_list = np.array(results_list)
df_results = pd.DataFrame({'wavelength': results_list[:,1], 
                   'gap': results_list[:,0], 
                   'a': results_list[:,3], 
                   'r': results_list[:,2], 
                   'finesse': results_list[:,5], 
                   'qfactor': results_list[:,6],
                   'FWHM': results_list[:,7],
                   'ERt': results_list[:,8],
                   'ERd': results_list[:,9],
                   'IL': results_list[:,10]})
del results_list

## Plots against gap

In [18]:
# Import the ColumnDataSource class
from bokeh.models import ColumnDataSource

tooltips = [
    ("Wavelength", "@wavelength mum"),
    ("(x,y)", "($x, $y)"),
]

pa = figure(plot_width=400, plot_height=400, tooltips=tooltips, tools="hover,crosshair,pan,box_zoom,wheel_zoom,undo,redo,reset,save",active_inspect="hover", active_drag="box_zoom", toolbar_location="right")
pa.xaxis.axis_label = "Gap (nm)"
pa.yaxis.axis_label = "a"

pr = figure(plot_width=400, plot_height=400, y_axis_type="log", tooltips=tooltips, tools="hover,crosshair,pan,box_zoom,wheel_zoom,undo,redo,reset,save",active_inspect="hover", active_drag="box_zoom", toolbar_location="right")
pr.xaxis.axis_label = "Gap (nm)"
pr.yaxis.axis_label = "r"

pFin = figure(plot_width=400, plot_height=400, tooltips=tooltips, tools="hover,crosshair,pan,box_zoom,wheel_zoom,undo,redo,reset,save",active_inspect="hover", active_drag="box_zoom", toolbar_location="right")
pFin.xaxis.axis_label = "Gap (nm)"
pFin.yaxis.axis_label = "Finesse"

pQ = figure(plot_width=400, plot_height=400, y_axis_type="log", tooltips=tooltips, tools="hover,crosshair,pan,box_zoom,wheel_zoom,undo,redo,reset,save",active_inspect="hover", active_drag="box_zoom", toolbar_location="right")
pQ.xaxis.axis_label = "Gap (nm)"
pQ.yaxis.axis_label = "Q-factor"

pFWHM = figure(plot_width=400, plot_height=400, tooltips=tooltips, tools="hover,crosshair,pan,box_zoom,wheel_zoom,undo,redo,reset,save",active_inspect="hover", active_drag="box_zoom", toolbar_location="right")
pFWHM.xaxis.axis_label = "Gap (nm)"
pFWHM.yaxis.axis_label = "FWHM (m)"

pERt = figure(plot_width=400, plot_height=400, tooltips=tooltips, tools="hover,crosshair,pan,box_zoom,wheel_zoom,undo,redo,reset,save",active_inspect="hover", active_drag="box_zoom", toolbar_location="right")
pERt.xaxis.axis_label = "Gap (nm)"
pERt.yaxis.axis_label = "ERt (dB)"

pERd = figure(plot_width=400, plot_height=400, tooltips=tooltips, tools="hover,crosshair,pan,box_zoom,wheel_zoom,undo,redo,reset,save",active_inspect="hover", active_drag="box_zoom", toolbar_location="right")
pERd.xaxis.axis_label = "Gap (nm)"
pERd.yaxis.axis_label = "ERd (dB)"

pIL = figure(plot_width=400, plot_height=400, y_axis_type="log", tooltips=tooltips, tools="hover,crosshair,pan,box_zoom,wheel_zoom,undo,redo,reset,save",active_inspect="hover", active_drag="box_zoom", toolbar_location="right")
pIL.xaxis.axis_label = "Gap (nm)"
pIL.yaxis.axis_label = "IL"

colors = itertools.cycle(viridis(len(wavelengths_toPlot)))

for wavelength, color in zip(wavelengths_toPlot, colors):
    to_plot_df = df_results.query('wavelength == @wavelength')
    
    # Convert dataframe to column data source
    src = ColumnDataSource(to_plot_df)

    pa.line(y='a', x='gap', line_width=2, color=color, source=src)
    pr.line(y='r', x='gap', line_width=2, color=color, source=src)
    pFin.line(y='finesse', x='gap', line_width=2, color=color, source=src)
    pQ.line(y='qfactor', x='gap', line_width=2, color=color, source=src)
    pERt.line(y='ERt', x='gap', line_width=2, color=color, source=src)
    pERd.line(y='ERd', x='gap', line_width=2, color=color, source=src)
    pFWHM.line(y='FWHM', x='gap', line_width=2, color=color, source=src)
    pIL.line(y='IL', x='gap', line_width=2, color=color, source=src)
    
pr.legend.location = "bottom_right"
pFin.legend.location = "top_left"
pQ.legend.location = "top_left"
pERt.legend.location = "bottom_left"
pERd.legend.location = "bottom_right"
pIL.legend.location = "bottom_right"

# Setup legend
for p in [pa, pr, pFin, pQ, pFWHM, pERt, pERd, pIL]:
    p.legend.visible=False
    
# Top legend
wavelength_spacing = wavelengths_toPlot[1] - wavelengths_toPlot[0]

def hex_to_rgb(value):
    """Return (red, green, blue) for the color given as #rrggbb."""
    value = value.lstrip('#')
    lv = len(value)
    return tuple(int(value[i:i + lv // 3], 16) for i in range(0, lv, lv // 3))

text = f'wavelength min = {np.min(wavelengths_toPlot):.2f} mum'
background = hex_to_rgb(viridis(len(wavelengths_toPlot))[0])
foreground_str = '37;'
background_str = '48;2;' + str(background[0]) + ';' + str(background[1]) + ';' + str(background[2]) + 'm'
min_wavelength_str = '\x1b[' + foreground_str + background_str + text + '\x1b[0m'

text = f'wavelength max = {np.max(wavelengths_toPlot):.2f} mum'
background = hex_to_rgb(viridis(len(wavelengths_toPlot))[-1])
foreground_str = '30;'
background_str = '48;2;' + str(background[0]) + ';' + str(background[1]) + ';' + str(background[2]) + 'm'
max_wavelength_str = '\x1b[' + foreground_str + background_str + text + '\x1b[0m'

print(f'Legend : ' + min_wavelength_str + ', ' + max_wavelength_str + ', ' +  f'spacing = {wavelength_spacing:.2f} nm')

show(row(pa,pr))
show(row(pFin,pQ))
show(row(pERt,pERd))
show(row(pFWHM,pIL))

C:\Users\simon\AppData\Local\Continuum\anaconda3\lib\site-packages\bokeh\models\plots.py:748: UserWarning: 
You are attempting to set `plot.legend.location` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with the 'legend' parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)
C:\Users\simon\AppData\Local\Continuum\anaconda3\lib\site-packages\bokeh\models\plots.py:748: UserWarning: 
You are attempting to set `plot.legend.visible` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with the 'legend' parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)


Legend : wavelength min = 1.50 mum, wavelength max = 1.60 mum, spacing = 0.01 nm


## Plots against wavelength

In [19]:
# Import the ColumnDataSource class
from bokeh.models import ColumnDataSource

tooltips = [
    ("Gap", "@gap nm"),
    ("(x,y)", "($x, $y)"),
]

pa = figure(plot_width=400, plot_height=400, tooltips=tooltips, tools="hover,crosshair,pan,box_zoom,wheel_zoom,undo,redo,reset,save",active_inspect="hover", active_drag="box_zoom", toolbar_location="right")
pa.xaxis.axis_label = "Wavelength (micron)"
pa.yaxis.axis_label = "a"

pr = figure(plot_width=400, plot_height=400, tooltips=tooltips, y_axis_type="log", tools="hover,crosshair,pan,box_zoom,wheel_zoom,undo,redo,reset,save",active_inspect="hover", active_drag="box_zoom", toolbar_location="right")
pr.xaxis.axis_label = "Wavelength (micron)"
pr.yaxis.axis_label = "r"

pFin = figure(plot_width=400, plot_height=400, tooltips=tooltips, tools="hover,crosshair,pan,box_zoom,wheel_zoom,undo,redo,reset,save",active_inspect="hover", active_drag="box_zoom", toolbar_location="right")
pFin.xaxis.axis_label = "Wavelength (micron)"
pFin.yaxis.axis_label = "Finesse"

pQ = figure(plot_width=400, plot_height=400, y_axis_type="log", tooltips=tooltips, tools="hover,crosshair,pan,box_zoom,wheel_zoom,undo,redo,reset,save",active_inspect="hover", active_drag="box_zoom", toolbar_location="right")
pQ.xaxis.axis_label = "Wavelength (micron)"
pQ.yaxis.axis_label = "Q-factor"

pFWHM = figure(plot_width=400, plot_height=400, tooltips=tooltips, tools="hover,crosshair,pan,box_zoom,wheel_zoom,undo,redo,reset,save",active_inspect="hover", active_drag="box_zoom", toolbar_location="right")
pFWHM.xaxis.axis_label = "Wavelength (micron)"
pFWHM.yaxis.axis_label = "FWHM (m)"

pERt = figure(plot_width=400, plot_height=400, tooltips=tooltips, tools="hover,crosshair,pan,box_zoom,wheel_zoom,undo,redo,reset,save",active_inspect="hover", active_drag="box_zoom", toolbar_location="right")
pERt.xaxis.axis_label = "Wavelength (micron)"
pERt.yaxis.axis_label = "ERt (dB)"

pERd = figure(plot_width=400, plot_height=400, tooltips=tooltips, tools="hover,crosshair,pan,box_zoom,wheel_zoom,undo,redo,reset,save",active_inspect="hover", active_drag="box_zoom", toolbar_location="right")
pERd.xaxis.axis_label = "Wavelength (micron)"
pERd.yaxis.axis_label = "ERd (dB)"

pIL = figure(plot_width=400, plot_height=400, y_axis_type="log", tooltips=tooltips, tools="hover,crosshair,pan,box_zoom,wheel_zoom,undo,redo,reset,save",active_inspect="hover", active_drag="box_zoom", toolbar_location="right")
pIL.xaxis.axis_label = "Wavelength (micron)"
pIL.yaxis.axis_label = "IL"

colors = itertools.cycle(viridis(len(gaps_toPlot)))

for gap, color in zip(gaps_toPlot, colors):
    to_plot_df = df_results.query('gap == @gap')
    
    # Convert dataframe to column data source
    src = ColumnDataSource(to_plot_df)

    pa.line(y='a', x='wavelength', line_width=2, color=color, source=src)
    pr.line(y='r', x='wavelength', line_width=2, color=color, source=src)
    pFin.line(y='finesse', x='wavelength', line_width=2, color=color, source=src)
    pQ.line(y='qfactor', x='wavelength', line_width=2, color=color, source=src)
    pERt.line(y='ERt', x='wavelength', line_width=2, color=color, source=src)
    pERd.line(y='ERd', x='wavelength', line_width=2, color=color, source=src)
    pFWHM.line(y='FWHM', x='wavelength', line_width=2, color=color, source=src)
    pIL.line(y='IL', x='wavelength', line_width=2, color=color, source=src)
    
pr.legend.location = "bottom_right"
pFin.legend.location = "top_left"
pQ.legend.location = "top_left"
pERt.legend.location = "bottom_left"
pERd.legend.location = "bottom_right"
pIL.legend.location = "bottom_right"

for p in [pa, pr, pFin, pQ, pFWHM, pERt, pERd, pIL]:
    p.legend.visible=False
    
# Top legend
gaps_spacing = gaps_toPlot[1] - gaps_toPlot[0]

def hex_to_rgb(value):
    """Return (red, green, blue) for the color given as #rrggbb."""
    value = value.lstrip('#')
    lv = len(value)
    return tuple(int(value[i:i + lv // 3], 16) for i in range(0, lv, lv // 3))

text = f'gap min = {np.min(gaps_toPlot):.2f} nm'
background = hex_to_rgb(viridis(len(gaps_toPlot))[0])
foreground_str = '37;'
background_str = '48;2;' + str(background[0]) + ';' + str(background[1]) + ';' + str(background[2]) + 'm'
min_gap_str = '\x1b[' + foreground_str + background_str + text + '\x1b[0m'

text = f'gap max = {np.max(gaps_toPlot):.2f} nm'
background = hex_to_rgb(viridis(len(gaps_toPlot))[-1])
foreground_str = '30;'
background_str = '48;2;' + str(background[0]) + ';' + str(background[1]) + ';' + str(background[2]) + 'm'
max_gap_str = '\x1b[' + foreground_str + background_str + text + '\x1b[0m'

print(f'Legend : ' + min_gap_str + ', ' + max_gap_str + ', ' +  f'spacing = {gaps_spacing:.2f} nm')

show(row(pa,pr))
show(row(pFin,pQ))
show(row(pERt,pERd))
show(row(pFWHM,pIL))

Legend : gap min = 200.00 nm, gap max = 500.00 nm, spacing = 6.12 nm
